# Image Segmentation using Compact Watershed Method

Produce segmented images on original dataset.

In [1]:
from skimage.io import imread
from skimage.segmentation import watershed, mark_boundaries
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage.filters import sobel
from PIL import Image
import matplotlib.pyplot as plt
import keras

In [2]:
def plot_image(img, title):
    plt.imshow(img), plt.title(title, size=20),  plt.axis('off') 

In [3]:
def compact_watershed(image_bytes):
    img = imread(image_bytes)
    #im = img.resize((32,32))
    #im = resize(img, (img.shape[0] // 4, img.shape[1] // 4), anti_aliasing=True)
    
    gradient = sobel(rgb2gray(img))
    segments_watershed = watershed(gradient, markers=15, compactness=0.00001)
    im = mark_boundaries(img, segments_watershed)
    #plot_image(im, '')
    return im

In [4]:
import numpy as np
import pandas as pd
from zipfile import ZipFile

label_dict = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 
              'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}

n_train = 50000, n_test = 10000
X_train = [[]]*n_train, Y_train = [[]]*n_train
X_test = [[]]*n_test, Y_test = [[]]*n_test
train_index = 0, test_index = 0

# open zip file for reading
with ZipFile('cfar-10.zip', 'r') as zip_data:
    content_list = zip_data.namelist()
    
    # read all images in the zip file and produce segmented images
    for file_path in content_list:
        path = file_path.split('/')
        label = label_dict.get(path[-2])
        img_bytes = zip_data.open(file_path)
        
        # apply 'compact watershed' segmentation
        img_ary = compact_watershed(img_bytes)
        
        if path[-3] == 'train':
            X_train[train_index] = img_ary
            Y_train[train_index] = label
            Y_train_list.append(label)
            train_index += 1
        else:
            X_test[test_index] = img_ary
            Y_test[test_index] = label
            Y_test_list.append(label)
            test_index += 1

In [5]:
# RGB values are between 0 and 255
# normalize RGB values to the scale between 0 and 1
X_train = np.asarray(X_train, dtype='float32')/255
X_test = np.asarray(X_test, dtype='float32')/255

In [6]:
Y_train = np.asarray(Y_train).reshape(-1,1)
Y_test = np.asarray(Y_test).reshape(-1,1)

In [7]:
# save np arry
np.save('data/X_train_cw.npy', X_train)
np.save('data/Y_train_cw.npy', Y_train)
np.save('data/X_test_cw.npy', X_test)
np.save('data/Y_test_cw.npy', Y_test)